<a href="https://colab.research.google.com/github/S-EGK/Contollers/blob/main/Sliding%20Mode%20Controller%20for%20Pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [ ]:
# importing libraries
from math import cos, sin
import numpy as np
from random import random
import scipy.signal as signal
import scipy.linalg as linalg
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.patches import Circle

## Parameters

### Simulation Parameters

In [ ]:
dt = 0.1
Tfinal = 40
Ts = dt
Tsteps = Tfinal/Ts # number of frames
frames = int(Tsteps)

### System Parameters

In [ ]:
# States
x1 = 0.01
x2 = 1

states = np.array([x1, x2])

### Datat Collection and Plot Parameters

In [ ]:
# Data Collectors
f_data = []
x1_data = []
x2_data = []

# Animation Parameters
rod_x = [0, sin(x1)]
rod_y = [0, -cos(x1)]

fig = plt.figure(figsize = (10,5))
ax1 = plt.subplot(1,1,1)
ax1.set_xlim((-10,10))
ax1.set_ylim((-3,3))

rod, = ax1.plot(rod_x, rod_y, 'b', lw = 2)
bob, = ax1.plot(sin(x1), -cos(x1), 'g', marker = 'o', ms = 10)

In [ ]:
def anipts(x1):
  rod_x = [0, sin(x1)]
  rod_y = [0, -cos(x1)]
  bob_x, bob_y = sin(x1), -cos(x1)

  return rod_x, rod_y, bob_x, bob_y

## Dynamics

In [ ]:
def dyna(t,y,f):
  x1 = y[0]
  x2 = y[1]
  u = f

  x1_dot = x2 + sin(x1)
  x2_dot = 2*x1*x1 + 3/2 * u

  return x1_dot, x2_dot

## Sliding Mode Controller

In [ ]:
def slid_ctrl(y):
  x1 = y[0]
  x2 = y[1]

  beta = 0.6
  a = 2
  s = a * x1 + x2
  e = 0.9

  s1 = np.absolute(s/e)

  if s1 <= 1:
    u = -(2/3)*(a*x2 + a*x1 + 2*x1*x1 + beta*s)
  elif s1 > 1:
    u = -(2/3)*(a*x2 + a*x1 + 2*x1*x1 + beta*np.sign(s)) 

  # u = -(2/3)*(a*x2 + a*x1 + 2*x1*x1 + beta*np.sign(s))

  return u

## Solver

In [ ]:
def step(states, dt):
  x1 = states[0]
  x2 = states[1]

  f = slid_ctrl(states)
  f_data.append(f)

  sol = solve_ivp(lambda t,y: dyna(t,y,f), [0,dt], [x1, x2], t_eval = np.linspace(0,dt,100))
  states = sol.y[:,-1].T

  return states

## Iterating

In [ ]:
def drawframe(i):
  global states
  states = step(states, dt)
  x1 = states[0]
  x2 = states[1]

  x1_data.append(states[0])
  x2_data.append(states[1])

  rod_x, rod_y, bob_x, bob_y = anipts(x1)

  rod.set_data(rod_x, rod_y)
  bob.set_data(bob_x, bob_y)

  return rod, bob

## Animation

In [ ]:
from matplotlib import animation
# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=frames, interval=40, blit=True)

In [ ]:
from IPython.display import HTML
HTML(anim.to_html5_video())

## Plots

In [ ]:
fig1 = plt.figure(figsize=(17,22), dpi=90)

ax1 = fig1.add_subplot(4, 3, 1)
ax1.set_xlabel('Time Step')
ax1.set_ylabel('x1')
ax1.set_title('State Plot of x1')
x11 = ax1.plot(range(len(x1_data)),x1_data)

ax2 = fig1.add_subplot(4, 3, 2)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('x2')
ax2.set_title('State Plot of x2')
x21 = ax2.plot(range(len(x2_data)),x2_data)

ax3 = fig1.add_subplot(4, 3, 3)
ax3.set_xlabel('x1')
ax3.set_ylabel('x2')
ax3.set_title('x1 - x2')
x1x21 = ax3.plot(x1_data,x2_data)

ax4 = fig1.add_subplot(4, 3, 4)
ax4.set_xlabel('Time Step')
ax4.set_ylabel('f')
ax4.set_title('f vs time step')
f1 = ax4.plot(range(len(f_data)),f_data)